In [1]:
import tensorflow as tf
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join
%matplotlib inline

In [2]:

def get_filename_queue(file_numbers, data_dir):
    # reconstruce filename for three source
    left_image_filename_list = []
    right_image_filename_list = []
    lidar_filename_list = []

    for i in range(len(file_numbers)):
        left_image_filename = data_dir +  file_numbers[i] + "left.png"
        right_image_filename = data_dir +  file_numbers[i] + "right.png"
        lidar_filename = data_dir + file_numbers[i] + ".txt"
        left_image_filename_list.append(left_image_filename)
        right_image_filename_list.append(right_image_filename)
        lidar_filename_list.append(lidar_filename)

    # list to tensor queue
    left_image_filename_queue = tf.train.string_input_producer(left_image_filename_list)
    right_image_filename_queue = tf.train.string_input_producer(right_image_filename_list)
    lidar_filename_queue = tf.train.string_input_producer(lidar_filename_list)

    return [left_image_filename_queue, right_image_filename_queue, lidar_filename_queue]


In [3]:
def read_data(left_image_filename_queue, right_image_filename_queue, lidar_filename_queue):

    # creat three class 
    class Left_Image_Record(object):
        pass
    Left_Image = Left_Image_Record()

    class Right_Image_Record(object):
        pass
    Right_Image = Right_Image_Record()

    class Lidar_Record(object):
        pass
    Lidar = Lidar_Record()

    # read and decode image    
    image_reader = tf.WholeFileReader()
    Left_Image.key, left_value = image_reader.read(left_image_filename_queue)
    # Left_Image.image = tf.image.decode_png(left_value)
    Left_Image.uint8image  = tf.reshape(tf.image.decode_png(left_value), [ORIGINAL_IMAGE_HEIGHT, ORIGINAL_IMAGE_WIDTH, IMAGE_DEPTH])

    Right_Image.key, right_value = image_reader.read(right_image_filename_queue)
    # Right_Image.image = tf.image.decode_png(right_value)
    Right_Image.uint8image  = tf.reshape(tf.image.decode_png(right_value), [ORIGINAL_IMAGE_HEIGHT, ORIGINAL_IMAGE_WIDTH, IMAGE_DEPTH])

    # read and decode lidar    
    lidar_reader = tf.TextLineReader()
    Lidar.key, lidar_value = lidar_reader.read(lidar_filename_queue)
    record_defaults = [[] for col in range(120)]
    lidar_samples = tf.decode_csv(lidar_value, record_defaults=record_defaults)
    Lidar.info = tf.pack(lidar_samples)

    return [Left_Image, Right_Image, Lidar]

In [4]:
def _generate_image_and_label_batch(left_image, right_image, lidar, min_queue_examples, batch_size, shuffle):
    """Construct a queued batch of images and labels.

    Args:
    left_image: 3-D Tensor of [height, width, 3] of type.float32.
    right_image: 3-D Tensor of [height, width, 3] of type.float32.
    lidar: 1-D Tensor of [lidar_size] type.float32
    min_queue_examples: int32, minimum number of samples to retain
      in the queue that provides of batches of examples.
    batch_size: Number of images per batch.
    shuffle: boolean indicating whether to use a shuffling queue.

    Returns:
    left_image_batch: Images. 4D tensor of [batch_size, height, width, 3] size.
    right_image_batch: Images. 4D tensor of [batch_size, height, width, 3] size.
    lidar: Labels. 2D tensor of [batch_size, lidar_size] size.
    """
    # Create a queue that shuffles the examples
    # min_queue_examples = 100
    num_preprocess_threads = 8
    if shuffle:
        left_batch, right_batch, lidar_batch = tf.train.shuffle_batch(
            [left_image, right_image, lidar],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size,
            min_after_dequeue=min_queue_examples)
    else:
        left_batch, right_batch, lidar_batch = tf.train.batch(
            [left_image, right_image, lidar],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.image_summary('left_images', left_batch)
    tf.image_summary('right_images', right_batch)

    return left_batch, right_batch, lidar_batch 

In [5]:
# only get serial number
def get_filenumber(mypath):
    # mypath = './data/data1/'
    data_lidar = sorted(glob.glob(mypath + '*txt'), key=os.path.basename)
    print (mypath)

    file_numbers = []
    for i in range(len(data_lidar)):
        number = data_lidar[i]
        number = number.replace(mypath,"")
        number = number.replace(".txt","")
        file_numbers.append(number)
    print ("number of sample: ", len(file_numbers))
    return file_numbers

In [6]:
data_dir = './data/data1/'
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 1
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 1

IMAGE_HEIGHT = 16
IMAGE_WIDTH = 64

ORIGINAL_IMAGE_HEIGHT = 128
ORIGINAL_IMAGE_WIDTH = 672
IMAGE_DEPTH = 3

In [ ]:
# Start a new session to show example output.
with tf.Session() as sess:
    # Required to get the filename matching to run.
    tf.initialize_all_variables().run()

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
#     # Get an image tensor and print its value.
    file_numbers = get_filenumber(data_dir)
    num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
    left_image_filename_queue, right_image_filename_queue, lidar_filename_queue = get_filename_queue(file_numbers, data_dir)
    Left_Image, Right_Image, Lidar = read_data(left_image_filename_queue, right_image_filename_queue, lidar_filename_queue)
#     left_image = tf.cast(Left_Image.uint8image, tf.float32)
#     right_image = tf.cast(Right_Image.uint8image, tf.float32)
    print (Left_Image.key)
    print (Right_Image.key)
    leftkey = sess.run(Right_Image.key)
    print (leftkey)
#     tf.Print(Left_Image.key, [Left_Image.key])
#     tf.Print(Right_Image.key, [Right_Image.key])
#     print (Lidar.key.eval())
#     image_tensor = sess.run(left_image) # read single image
#     print image_tensor.shape
#     plt.imshow(image_tensor)

    # Get lidar info
#     lidar_value = sess.run(lidar_label)
#     lidar_np = np.fromstring(lidar_string, sep='\n')
#     print(lidar_label.eval())
    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)

./data/data1/
('number of sample: ', 1219)
Tensor("ReaderRead:0", shape=(), dtype=string)
Tensor("ReaderRead_1:0", shape=(), dtype=string)
